*Copyright 2024 The Penzai Authors.*

*Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License.
You may obtain a copy of the License at*

> http://www.apache.org/licenses/LICENSE-2.0

*Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or  implied.
See the License for the specific language governing permissions and
limitations under the License.*

---

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google-deepmind/penzai/blob/main/notebooks/selectors.ipynb) [![Open in Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/google-deepmind/penzai/blob/main/notebooks/selectors.ipynb)

# Selectors and Selections

Penzai is designed to make it easy to make targeted modifications to models and parameter trees. For this purpose, Penzai provides a powerful "selection" system to allow you to identify, visualize, and modify arbitrary JAX PyTrees. In short, `pz.select` enables `.at[...].set(...)`-style modification for arbitrary PyTree types.

This notebook describes the basics of Penzai's selection object and shows how you can use it to make a variety of modifications.


## Setup

We'll start by setting up the environment.

### Imports

To run this notebook, you need a Python environment with `penzai` and its dependencies installed.

In Colab or Kaggle, you can install it using the following command:

In [1]:
try:
  import penzai
except ImportError:
  !pip install penzai[notebook]

In [2]:
from __future__ import annotations
import typing
import traceback

import jax
import jax.numpy as jnp

In [3]:
import penzai
from penzai import pz

In [4]:
from penzai.example_models import simple_mlp

### Setting up Penzai

For this tutorial, we'll enable Treescope (Penzai's pretty-printer) as the default IPython pretty-printer. This is recommended when using Penzai in an interactive environment.

In [5]:
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.ts.register_context_manager_magic()

## What are selectors?

If you're familiar with JAX, you probably know that JAX arrays are immutable. This maens that it's not possible to directly assign to elements of a JAX array:

In [6]:
array = jnp.zeros((10, 10))
array

# jax.Array float32(10, 10) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:100
  Array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [7]:
try:
  array[1, 2] = 42
except TypeError:
  import traceback
  traceback.print_exc()

Traceback (most recent call last):
  File "<ipython-input-7-5c112bb0ee90>", line 2, in <cell line: 1>
    array[1, 2] = 42
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/numpy/array_methods.py", line 278, in _unimplemented_setitem
    raise TypeError(msg.format(type(self)))
TypeError: '<class 'jaxlib.xla_extension.ArrayImpl'>' object does not support item assignment. JAX arrays are immutable. Instead of ``x[idx] = y``, use ``x = x.at[idx].set(y)`` or another .at[] method: https://jax.readthedocs.io/en/latest/_autosummary/jax.numpy.ndarray.at.html


Instead, to modify a JAX array you can use the special `.at` property, which returns a modified copy of the original array with changes at the location you specified:

In [8]:
modified_array = array.at[1, 2].set(42)
modified_array

# jax.Array float32(10, 10) ≈0.42 ±4.2 [≥0.0, ≤4.2e+01] zero:99 nonzero:1
  Array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0., 42.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]], dtype=float32)

JAX also introduces the concept of [PyTrees](https://jax.readthedocs.io/en/latest/pytrees.html): nested containers of data that JAX knows how to traverse. Most JAX transformations are designed to work with functions that take PyTrees as input and return PyTrees as output.

Although JAX does treat dictionaries and lists as PyTrees, JAX usually assumes that those trees will not be mutated, and expects the user to write code in a functional style, avoiding mutating its arguments directly.

**`pz.select` enables `.at[...].set(...)`-style modification for arbitrary PyTrees**, making it possible to perform complex path-based and type-driven modifications to the objects you're already using:

In [9]:
my_nested_object = {
    "a": 1,
    "b": jnp.arange(10),
    "c": [
        {"value": jnp.arange(12)},
        {"value": jnp.zeros([7])},
        {"value": 3},
    ]
}

In [10]:
(
    pz.select(my_nested_object)
      .at(lambda root: root["c"])
      .at_instances_of(jax.typing.ArrayLike)
      .apply(lambda value: value + 100)
)

{'a': 1,
 'b': <jax.Array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>,
 'c': [{'value': <jax.Array([100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111], dtype=int32)>},
  {'value': <jax.Array([100., 100., 100., 100., 100., 100., 100.], dtype=float32)>},
  {'value': 103}]}

Selectors are first-class objects, defined in `penzai.core.selectors` but aliased to `pz.select` and `pz.Selection`. Internally, they are ordinary PyTrees:


In [11]:
my_selection = (
    pz.select(my_nested_object)
      .at(lambda root: root["c"])
      .at_instances_of(jax.typing.ArrayLike)
)

pz.show(my_selection)

TreescopeRenderingFigure(child=PreventWordWrap(child=Siblings(children=(FoldableTreeNodeImpl(contents=Siblings(children=(IndentedChildren(children=[Siblings(children=(PyTreeNodeFieldName(child=Text(text='selected_by_path')), Text(text='='), WithDynamicSharedPip(child=FoldableTreeNodeImpl(contents=Siblings(children=(StringCopyButton(copy_string='(lambda root: root.selected_by_path)', annotation='Copy path: '), IndentedChildren(children=[Siblings(children=(FoldableTreeNodeImpl(contents=Siblings(children=(IndentedChildren(children=[Siblings(children=(FoldableTreeNodeImpl(contents=Siblings(children=(IndentedChildren(children=[Siblings(children=(Text(text='key'), Text(text='='), HyperlinkTarget(child=StringLiteralColor(child=Text(text="'c'")), keypath=None), FoldCondition(collapsed=EmptyPart(), expanded=Text(text=','))))]), Text(text=')'))), label=HyperlinkTarget(child=Siblings(children=(RoundtripCondition(roundtrip=QualifiedTypeNameSpanGroup(child=Text(text='jax.tree_util.')), not_roundtrip=EmptyPart()), Text(text='DictKey'), Text(text='('))), keypath=None), expand_state=<ExpandState.COLLAPSED: 'collapsed'>), Text(text=','), FoldCondition(collapsed=Text(text=' '), expanded=EmptyPart()))), Siblings(children=(FoldableTreeNodeImpl(contents=Siblings(children=(IndentedChildren(children=[Siblings(children=(Text(text='idx'), Text(text='='), HyperlinkTarget(child=NumberColor(child=Text(text='0')), keypath=None), FoldCondition(collapsed=EmptyPart(), expanded=Text(text=','))))]), Text(text=')'))), label=HyperlinkTarget(child=Siblings(children=(RoundtripCondition(roundtrip=QualifiedTypeNameSpanGroup(child=Text(text='jax.tree_util.')), not_roundtrip=EmptyPart()), Text(text='SequenceKey'), Text(text='('))), keypath=None), expand_state=<ExpandState.COLLAPSED: 'collapsed'>), Text(text=','), FoldCondition(collapsed=Text(text=' '), expanded=EmptyPart()))), Siblings(children=(FoldableTreeNodeImpl(contents=Siblings(children=(IndentedChildren(children=[Siblings(children=(Text(text='key'), Text(text='='), HyperlinkTarget(child=StringLiteralColor(child=Text(text="'value'")), keypath=None), FoldCondition(collapsed=EmptyPart(), expanded=Text(text=','))))]), Text(text=')'))), label=HyperlinkTarget(child=Siblings(children=(RoundtripCondition(roundtrip=QualifiedTypeNameSpanGroup(child=Text(text='jax.tree_util.')), not_roundtrip=EmptyPart()), Text(text='DictKey'), Text(text='('))), keypath=None), expand_state=<ExpandState.COLLAPSED: 'collapsed'>), FoldCondition(collapsed=EmptyPart(), expanded=Text(text=','))))]), Text(text=')'))), label=HyperlinkTarget(child=Text(text='('), keypath=None), expand_state=<ExpandState.COLLAPSED: 'collapsed'>), Text(text=':'), FoldCondition(collapsed=Text(text=' '), expanded=EmptyPart()), IndentedChildren(children=[Siblings(children=(RoundtripCondition(roundtrip=WithBlockColor(child=FoldableTreeNodeImpl(contents=Siblings(children=(ColoredBorderIndentedChildren(children=[Siblings(children=(Text(text='original_repr'), Text(text='='), HyperlinkTarget(child=StringLiteralColor(child=Text(text="'Array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int32)'")), keypath=(GetAttrKey(name='original_repr'),)), Text(text=','), FoldCondition(collapsed=Text(text=' '), expanded=EmptyPart()), FoldCondition(collapsed=EmptyPart(), expanded=Siblings(children=(StringCopyButton(copy_string='(lambda root: root.original_repr)', annotation='Copy path: '),))))), Siblings(children=(Text(text='original_id'), Text(text='='), HyperlinkTarget(child=NumberColor(child=Text(text='97072186721616')), keypath=(GetAttrKey(name='original_id'),)), Text(text=','), FoldCondition(collapsed=Text(text=' '), expanded=EmptyPart()), FoldCondition(collapsed=EmptyPart(), expanded=Siblings(children=(StringCopyButton(copy_string='(lambda root: root.original_id)', annotation='Copy path: '),))))), Siblings(children=(Text(text='original_type'), Text(text='='), HyperlinkTarget(child=FoldableTreeNodeImpl(contents=FoldCondition(collapsed=EmptyPart(), expanded=IndentedChildren(c

By default, though, selections are rendered in a way that emphasizes what you've selected, so you can tell at a glance that you're making the changes you expect to:

In [12]:
my_selection

Selection(
  selected_by_path=OrderedDict({
    (DictKey(key='c'), SequenceKey(idx=0), DictKey(key='value')):
      <jax.Array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int32)>,
    (DictKey(key='c'), SequenceKey(idx=1), DictKey(key='value')):
      <jax.Array([0., 0., 0., 0., 0., 0., 0.], dtype=float32)>,
    (DictKey(key='c'), SequenceKey(idx=2), DictKey(key='value')):
      3,
  }),
  remainder={'a': 1, 'b': <jax.Array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>, 'c': [{'value': SelectionHole(path=(DictKey(key='c'), SequenceKey(idx=0), DictKey(key='value')))}, {'value': SelectionHole(path=(DictKey(key='c'), SequenceKey(idx=1), DictKey(key='value')))}, {'value': SelectionHole(path=(DictKey(key='c'), SequenceKey(idx=2), DictKey(key='value')))}]},
)

(This fancy rendering mode is triggered when you display a single selection after enabling `pz.ts.register_as_default()`, or when you call `.show_selection()` on a selection object.)

## Building selections

The penzai selector API is designed around *method chaining*. You generally start by creating a trivial selection containing only a single object, the root object, using `pz.select`:

In [13]:
pz.select(my_nested_object)

Selection(
  selected_by_path=OrderedDict({
    (): {
      'a': 1,
      'b': <jax.Array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>,
      'c': [
        {
          'value': <jax.Array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int32)>,
        },
        {
          'value': <jax.Array([0., 0., 0., 0., 0., 0., 0.], dtype=float32)>,
        },
        {'value': 3},
      ],
    },
  }),
  remainder=SelectionHole(path=()),
)

You can also use the `.select()` attribute if you know your root object is a subclass of `pz.Struct`:

In [14]:
@pz.pytree_dataclass
class MyStruct(pz.Struct):
  foo: typing.Any

MyStruct(4).select()

Selection(
  selected_by_path=OrderedDict({(): MyStruct(foo=4)}),
  remainder=SelectionHole(path=()),
)

You then call methods on the selection to *refine* it to a subset of the currently selected values. These methods all operate relative to the currently selected object, so if you chain them, each method's output is restricted to the values selected by the previous method.

### Selecting with a function
If you want to select a specific part of a tree, you can use `selection.at(...)`. The `.at` method takes as input a *function* that extracts the part you want to select, and returns a new selection that selects the extracted part:

In [15]:
pz.select(my_nested_object).at(lambda root: root["c"])

Selection(
  selected_by_path=OrderedDict({
    (DictKey(key='c'),):
      [
        {
          'value': <jax.Array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int32)>,
        },
        {'value': <jax.Array([0., 0., 0., 0., 0., 0., 0.], dtype=float32)>},
        {'value': 3},
      ],
  }),
  remainder={
    'a': 1,
    'b': <jax.Array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>,
    'c': SelectionHole(path=(DictKey(key='c'),)),
  },
)

In [16]:
pz.select(my_nested_object).at(lambda root: root["c"][1]["value"])

Selection(
  selected_by_path=OrderedDict({
    (DictKey(key='c'), SequenceKey(idx=1), DictKey(key='value')):
      <jax.Array([0., 0., 0., 0., 0., 0., 0.], dtype=float32)>,
  }),
  remainder={
    'a': 1,
    'b': <jax.Array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>,
    'c': [
      {
        'value': <jax.Array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int32)>,
      },
      {'value': SelectionHole(path=(DictKey(key='c'), SequenceKey(idx=1), DictKey(key='value')))},
      {'value': 3},
    ],
  },
)

In [17]:
pz.select(my_nested_object).at(lambda root: root["b"])

Selection(
  selected_by_path=OrderedDict({
    (DictKey(key='b'),):
      <jax.Array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>,
  }),
  remainder={
    'a': 1,
    'b': SelectionHole(path=(DictKey(key='b'),)),
    'c': [
      {
        'value': <jax.Array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int32)>,
      },
      {'value': <jax.Array([0., 0., 0., 0., 0., 0., 0.], dtype=float32)>},
      {'value': 3},
    ],
  },
)

Later `at` calls in a chain are relative to the currently selected part:

In [18]:
(
    pz.select(my_nested_object)
    .at(lambda root: (root["c"][0], root["c"][1]))
    .at(lambda subtree: subtree["value"])
)

Selection(
  selected_by_path=OrderedDict({
    (DictKey(key='c'), SequenceKey(idx=0), DictKey(key='value')):
      <jax.Array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int32)>,
    (DictKey(key='c'), SequenceKey(idx=1), DictKey(key='value')):
      <jax.Array([0., 0., 0., 0., 0., 0., 0.], dtype=float32)>,
  }),
  remainder={
    'a': 1,
    'b': <jax.Array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>,
    'c': [{'value': SelectionHole(path=(DictKey(key='c'), SequenceKey(idx=0), DictKey(key='value')))}, {'value': SelectionHole(path=(DictKey(key='c'), SequenceKey(idx=1), DictKey(key='value')))}, {'value': 3}],
  },
)

There are a few restrictions on the function that is used to select a subtree. In particular:
- your function can't depend on the actual value passed in, only on its PyTree structure,
- your function should return a single node or tuple of nodes from the PyTree.

Internally, `Selection.at` is implemented using [`equinox.tree_at`](https://docs.kidger.site/equinox/api/manipulation/#equinox.tree_at), which takes care of most of the heavy lifting.

### Selecting by type

If you want to select all subtrees with a particular type, you can use `Selection.at_instances_of`:

In [19]:
pz.select(my_nested_object).at_instances_of(int)

Selection(
  selected_by_path=OrderedDict({
    (DictKey(key='a'),):
      1,
    (DictKey(key='c'), SequenceKey(idx=2), DictKey(key='value')):
      3,
  }),
  remainder={
    'a': SelectionHole(path=(DictKey(key='a'),)),
    'b': <jax.Array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>,
    'c': [{'value': <jax.Array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int32)>}, {'value': <jax.Array([0., 0., 0., 0., 0., 0., 0.], dtype=float32)>}, {'value': SelectionHole(path=(DictKey(key='c'), SequenceKey(idx=2), DictKey(key='value')))}],
  },
)

This selects any subtree for which `isinstance(subtree, requested_type)` evaluates to True.

Note that selections cannot be nested, so this only selects the *outermost* value with the given type:

In [20]:
pz.select(my_nested_object).at_instances_of(dict)

Selection(
  selected_by_path=OrderedDict({
    (): {
      'a': 1,
      'b': <jax.Array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>,
      'c': [
        {
          'value': <jax.Array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int32)>,
        },
        {
          'value': <jax.Array([0., 0., 0., 0., 0., 0., 0.], dtype=float32)>,
        },
        {'value': 3},
      ],
    },
  }),
  remainder=SelectionHole(path=()),
)

### Selecting by condition
More generally, you can select all subtrees for which a function evaluates to true:

In [21]:
(
    pz.select(my_nested_object)
    .at_subtrees_where(
        lambda subtree: isinstance(subtree, jax.Array) and subtree.size <= 10)
)

Selection(
  selected_by_path=OrderedDict({
    (DictKey(key='b'),):
      <jax.Array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>,
    (DictKey(key='c'), SequenceKey(idx=1), DictKey(key='value')):
      <jax.Array([0., 0., 0., 0., 0., 0., 0.], dtype=float32)>,
  }),
  remainder={
    'a': 1,
    'b': SelectionHole(path=(DictKey(key='b'),)),
    'c': [{'value': <jax.Array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int32)>}, {'value': SelectionHole(path=(DictKey(key='c'), SequenceKey(idx=1), DictKey(key='value')))}, {'value': 3}],
  },
)

In fact, `at_instances_of` is a thin wrapper around `at_subtrees_where`. There's another convenience method for finding values equal to a sentinel value:

In [22]:
(
    pz.select(my_nested_object)
    .at_equal_to(1)
)

Selection(
  selected_by_path=OrderedDict({(DictKey(key='a'),): 1}),
  remainder={
    'a': SelectionHole(path=(DictKey(key='a'),)),
    'b': <jax.Array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>,
    'c': [
      {
        'value': <jax.Array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int32)>,
      },
      {
        'value': <jax.Array([0., 0., 0., 0., 0., 0., 0.], dtype=float32)>,
      },
      {'value': 3},
    ],
  },
)

In [23]:
(
    pz.select({"foo": "foo", "bar": [1, 2, 3]}).at_equal_to([1, 2, 3])
)

Selection(
  selected_by_path=OrderedDict({(DictKey(key='bar'),): [1, 2, 3]}),
  remainder={
    'bar': SelectionHole(path=(DictKey(key='bar'),)),
    'foo': 'foo',
  },
)

### Selecting by JAX keypath

PyTree nodes in JAX are associated with key paths identifying their location in the tree:

In [24]:
jax.tree_util.tree_map_with_path(lambda key, node: key, my_nested_object)

{'a': (DictKey(key='a'),),
 'b': (DictKey(key='b'),),
 'c': [{'value': (DictKey(key='c'), SequenceKey(idx=0), DictKey(key='value'))},
  {'value': (DictKey(key='c'), SequenceKey(idx=1), DictKey(key='value'))},
  {'value': (DictKey(key='c'), SequenceKey(idx=2), DictKey(key='value'))}]}

You can directly select nodes based on their JAX keypath:

In [25]:
pz.select(my_nested_object).at_keypaths([
    (jax.tree_util.DictKey(key='c'), jax.tree_util.SequenceKey(idx=2), jax.tree_util.DictKey(key='value')),
    (jax.tree_util.DictKey(key='c'), jax.tree_util.SequenceKey(idx=0)),
])

Selection(
  selected_by_path=OrderedDict({
    (DictKey(key='c'), SequenceKey(idx=0)):
      {
        'value': <jax.Array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int32)>,
      },
    (DictKey(key='c'), SequenceKey(idx=2), DictKey(key='value')):
      3,
  }),
  remainder={
    'a': 1,
    'b': <jax.Array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>,
    'c': [SelectionHole(path=(DictKey(key='c'), SequenceKey(idx=0))), {'value': <jax.Array([0., 0., 0., 0., 0., 0., 0.], dtype=float32)>}, {'value': SelectionHole(path=(DictKey(key='c'), SequenceKey(idx=2), DictKey(key='value')))}],
  },
)

(You might have noticed that the fancy rendering for Selection objects is actually written in terms of `at_keypaths`, and exposes the keypaths for the currently selected nodes if you expand the last line!)

### Selecting based on PyTree structure

You can also build selections based on PyTree children or PyTree leaves:

In [26]:
pz.select(my_nested_object).at_children()

Selection(
  selected_by_path=OrderedDict({
    (DictKey(key='a'),):
      1,
    (DictKey(key='b'),):
      <jax.Array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>,
    (DictKey(key='c'),):
      [{'value': <jax.Array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int32)>}, {'value': <jax.Array([0., 0., 0., 0., 0., 0., 0.], dtype=float32)>}, {'value': 3}],
  }),
  remainder={'a': SelectionHole(path=(DictKey(key='a'),)), 'b': SelectionHole(path=(DictKey(key='b'),)), 'c': SelectionHole(path=(DictKey(key='c'),))},
)

In [27]:
pz.select(my_nested_object).at_pytree_leaves()

Selection(
  selected_by_path=OrderedDict({(DictKey(key='a'),): 1, (DictKey(key='b'),): <jax.Array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>, (DictKey(key='c'), SequenceKey(idx=0), DictKey(key='value')): <jax.Array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int32)>, (DictKey(key='c'), SequenceKey(idx=1), DictKey(key='value')): <jax.Array([0., 0., 0., 0., 0., 0., 0.], dtype=float32)>, (DictKey(key='c'), SequenceKey(idx=2), DictKey(key='value')): 3}),
  remainder={
    'a': SelectionHole(path=(DictKey(key='a'),)),
    'b': SelectionHole(path=(DictKey(key='b'),)),
    'c': [
      {
        'value': SelectionHole(
          path=(DictKey(key='c'), SequenceKey(idx=0), DictKey(key='value')),
        ),
      },
      {
        'value': SelectionHole(path=(DictKey(key='c'), SequenceKey(idx=1), DictKey(key='value'))),
      },
      {
        'value': SelectionHole(path=(DictKey(key='c'), SequenceKey(idx=2), DictKey(key='value'))),
      },
    ],
  },
)

Note that some PyTree nodes, like `None` or the empty tuple `()`, don't have any children and are ignored by `jax.tree_util.tree_map`, so they won't be selected by `.at_pytree_leaves`:

In [28]:
pz.select([1, 2, (), 3, None, 4]).at_pytree_leaves()

Selection(
  selected_by_path=OrderedDict({(SequenceKey(idx=0),): 1, (SequenceKey(idx=1),): 2, (SequenceKey(idx=3),): 3, (SequenceKey(idx=5),): 4}),
  remainder=[
    SelectionHole(path=(SequenceKey(idx=0),)),
    SelectionHole(path=(SequenceKey(idx=1),)),
    (),
    SelectionHole(path=(SequenceKey(idx=3),)),
    None,
    SelectionHole(path=(SequenceKey(idx=5),)),
  ],
)

If you want to select these too, you can use `.at_childless`:

In [29]:
pz.select([1, 2, (), 3, None, 4]).at_childless()

Selection(
  selected_by_path=OrderedDict({(SequenceKey(idx=0),): 1, (SequenceKey(idx=1),): 2, (SequenceKey(idx=2),): (), (SequenceKey(idx=3),): 3, (SequenceKey(idx=4),): None, (SequenceKey(idx=5),): 4}),
  remainder=[
    SelectionHole(path=(SequenceKey(idx=0),)),
    SelectionHole(path=(SequenceKey(idx=1),)),
    SelectionHole(path=(SequenceKey(idx=2),)),
    SelectionHole(path=(SequenceKey(idx=3),)),
    SelectionHole(path=(SequenceKey(idx=4),)),
    SelectionHole(path=(SequenceKey(idx=5),)),
  ],
)

### Filtering selections
Given an existing set of selected nodes, you can choose to filter down to only a subset that match a criterion:

In [30]:
pz.select(list(range(20))).at_instances_of(int).where(lambda x: x % 2 == 0)

Selection(
  selected_by_path=OrderedDict({(SequenceKey(idx=0),): 0, (SequenceKey(idx=2),): 2, (SequenceKey(idx=4),): 4, (SequenceKey(idx=6),): 6, (SequenceKey(idx=8),): 8, (SequenceKey(idx=10),): 10, (SequenceKey(idx=12),): 12, (SequenceKey(idx=14),): 14, (SequenceKey(idx=16),): 16, (SequenceKey(idx=18),): 18}),
  remainder=[SelectionHole(path=(SequenceKey(idx=0),)), 1, SelectionHole(path=(SequenceKey(idx=2),)), 3, SelectionHole(path=(SequenceKey(idx=4),)), 5, SelectionHole(path=(SequenceKey(idx=6),)), 7, SelectionHole(path=(SequenceKey(idx=8),)), 9, SelectionHole(path=(SequenceKey(idx=10),)), 11, SelectionHole(path=(SequenceKey(idx=12),)), 13, SelectionHole(path=(SequenceKey(idx=14),)), 15, SelectionHole(path=(SequenceKey(idx=16),)), 17, SelectionHole(path=(SequenceKey(idx=18),)), 19],
)

You can also choose to keep only the `n`th node that you've selected (e.g. the third integer in the collection), zero-indexed:

In [31]:
pz.select([1, "a", 2, "b", 3, "c", 4, "d"]).at_instances_of(int).pick_nth_selected(2)

Selection(
  selected_by_path=OrderedDict({(SequenceKey(idx=4),): 3}),
  remainder=[
    1,
    'a',
    2,
    'b',
    SelectionHole(path=(SequenceKey(idx=4),)),
    'c',
    4,
    'd',
  ],
)

### Inverting selections
Selections can be inverted, which produces a new Selection which selects every PyTree node that did NOT contain any of the selected nodes:

In [32]:
selection = pz.select([1, "a", 2, "b", 3, "c", 4, "d"]).at_instances_of(int).pick_nth_selected(2)

print("Original:")
selection.show_selection()
print("Inverted:")
selection.invert().show_selection()

Original:


Inverted:


In [33]:
selection = pz.select(my_nested_object).at_keypaths([
    (jax.tree_util.DictKey(key='c'), jax.tree_util.SequenceKey(idx=2), jax.tree_util.DictKey(key='value')),
    (jax.tree_util.DictKey(key='c'), jax.tree_util.SequenceKey(idx=0)),
])

print("Original:")
selection.show_selection()
print("Inverted:")
selection.invert().show_selection()

Original:


Inverted:


### Building more complex selections
For highly dynamic selections, you can drop down to lower-level manipulation.

Selections provide a `.refine` method, which runs a function on each of the currently selected values, takes the `Selection` returned by that function, and then takes the disjoint union of them. This can be useful for dynamic input-dependent branching:

In [34]:
# Select field "a" of Foo and field "b" of Bar

@pz.pytree_dataclass
class Foo(pz.Struct):
  a: typing.Any
  b: typing.Any

@pz.pytree_dataclass
class Bar(pz.Struct):
  a: typing.Any
  b: typing.Any

def refine_fn(value):
  if isinstance(value, Foo):
    return value.select().at(lambda x: x.a)
  else:
    return value.select().at(lambda x: x.b)

In [35]:
pz.select([
    Foo(1, 2),
    Bar(3, 4),
    Foo(5, 6),
    Bar(7, 8),
    None,
]).at_instances_of((Foo, Bar)).refine(refine_fn)

Selection(
  selected_by_path=OrderedDict({(SequenceKey(idx=0), GetAttrKey(name='a')): 1, (SequenceKey(idx=1), GetAttrKey(name='b')): 4, (SequenceKey(idx=2), GetAttrKey(name='a')): 5, (SequenceKey(idx=3), GetAttrKey(name='b')): 8}),
  remainder=[
    Foo(
      a=SelectionHole(path=(SequenceKey(idx=0), GetAttrKey(name='a'))),
      b=2,
    ),
    Bar(
      a=3,
      b=SelectionHole(path=(SequenceKey(idx=1), GetAttrKey(name='b'))),
    ),
    Foo(
      a=SelectionHole(path=(SequenceKey(idx=2), GetAttrKey(name='a'))),
      b=6,
    ),
    Bar(a=7, b=SelectionHole(path=(SequenceKey(idx=3), GetAttrKey(name='b')))),
    None,
  ],
)

## Inspecting selections and retrieving values

Once you have a selection, there are various ways to inspect them and their contents.

### Looking at selections
You can view a pretty-printed version of the selection using `.show_selection()` (the same pretty-printing we've been using throughout this notebook):

In [36]:
pz.select(my_nested_object).at_children().show_selection()

You can also view the selected values without the selection annotations using `.show_value()`. This is similar to just rendering the root object with treescope, but it automatically expands so that the nodes you selected are visible, and collapses all other nodes.

In [37]:
stuff = (
    [[["haystack"] * 5] * 5] * 5
    + [[["haystack"] * 5] * 5 + [["haystack", "needle", "haystack", "haystack", "haystack"]]]
    + [[["haystack"] * 5] * 5] * 5
)

pz.select(stuff).at_equal_to("needle").show_value()

### Counting selected objects
Selections have `count` and `is_empty` methods for inspecting their size:

In [38]:
pz.select([1, 2]).at_instances_of(str).count()

0

In [39]:
pz.select([1, 2]).at_instances_of(str).is_empty()

True

In [40]:
pz.select([1, 2]).at_instances_of(int).count()

2

In [41]:
pz.select([1, 2]).at_instances_of(int).is_empty()

False

If you already know how many objects should be selected, you can also add an assertion in the middle of a chain:

In [42]:
pz.select([1, 2]).at_instances_of(int).assert_count_is(2).apply(lambda x: x + 1)

[2, 3]

### Retrieving values

You can extract all of the selected values using `.get_sequence()`:

In [43]:
selection = (
    pz.select(my_nested_object)
    .at_subtrees_where(
        lambda subtree: isinstance(subtree, jax.Array) and subtree.size <= 10)
)

selection.get_sequence()

(<jax.Array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>,
 <jax.Array([0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)

Or get them in a dictionary form with `get_by_path()` (equivalent to just accessing the `.selected_by_path` attribute):

In [44]:
selection.get_by_path()

OrderedDict([((DictKey(key='b'),),
              <jax.Array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>),
             ((DictKey(key='c'), SequenceKey(idx=1), DictKey(key='value')),
              <jax.Array([0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)])

If you know there's exactly one value, you can just call `.get()`

In [45]:
pz.select(my_nested_object).at(lambda root: root["c"][0]).get()

{'value': <jax.Array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int32)>}

You can also get the selected objects in a dictionary form by accessing the `selected_by_path` attribute, which is how selections store their selected nodes internally:

In [46]:
(
    pz.select(my_nested_object)
    .at_subtrees_where(
        lambda subtree: isinstance(subtree, jax.Array) and subtree.size <= 10
    )
).selected_by_path

OrderedDict([((DictKey(key='b'),),
              <jax.Array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>),
             ((DictKey(key='c'), SequenceKey(idx=1), DictKey(key='value')),
              <jax.Array([0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)])

### Partitioning PyTrees

You can use `.partition()` to split a selected object into two parts, one containing only the selected subtrees, and one containing everything else:

In [47]:
selected, rest = (
    pz.select(my_nested_object)
    .at_subtrees_where(
        lambda subtree: isinstance(subtree, jax.Array) and subtree.size <= 10)
).partition()

In [48]:
selected

{'a': NotInThisPartition(),
 'b': <jax.Array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>,
 'c': [NotInThisPartition(),
  {'value': <jax.Array([0., 0., 0., 0., 0., 0., 0.], dtype=float32)>},
  NotInThisPartition()]}

In [49]:
rest

{'a': 1,
 'b': NotInThisPartition(),
 'c': [{'value': <jax.Array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int32)>},
  {'value': NotInThisPartition()},
  {'value': 3}]}

You can then process the two parts independently, and then recombine them into a single object using `pz.combine`:

In [50]:
pz.combine(
    jax.tree_util.tree_map(lambda x: x + 100, selected),
    rest,
)

{'a': 1,
 'b': <jax.Array([100, 101, 102, 103, 104, 105, 106, 107, 108, 109], dtype=int32)>,
 'c': [{'value': <jax.Array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int32)>},
  {'value': <jax.Array([100., 100., 100., 100., 100., 100., 100.], dtype=float32)>},
  {'value': 3}]}

Partitioning and combining are inspired by `equinox.partition` and `equinox.combine`. If you're already familiar with those, the main differences are:
- You generally don't need to use `partition` and `combine` when running JAX transformations like `jax.jit`. By convention, Penzai models store all of their static metadata in `dataclasses.field(metadata={"pytree_node": False})` fields which are not part of the PyTree traversal, which means partitioning isn't as strictly necessary as it is in equinox workflows. Instead, partitioning is primarily useful if you want to apply different logic to two sets of leaves, e.g. taking a gradient only with respect to a specific subset of parameters, or defining different shardings for different array subsets.
- Penzai uses a specific sentinel `NotInThisPartition()` to identify removed nodes, rather than `None`.
- Penzai partitions are built to support manipulation at the subtree level, rather than at the leaf level. It's OK to build and combine partitions even when neither partition is a strict PyTree prefix of the other, as long as the overlapping parts don't conflict.
- Penzai partitions are always created with selectors, rather than being built by a standalone function.


## Modifying selected values

An important feature of selections is that they allow you to perform detailed modifications to (copies of) large trees. Selections expose a number of methods for this purpose.

### Replacing selected values
The simplest modification you can make is to replace each selected subtree with another subtree or value:

In [51]:
pz.select(my_nested_object).at_instances_of(jax.Array).set("hello world!")

{'a': 1,
 'b': 'hello world!',
 'c': [{'value': 'hello world!'}, {'value': 'hello world!'}, {'value': 3}]}

You can optionally provide different values for each selected node:

In [52]:
pz.select(my_nested_object).at_instances_of(jax.Array).set_sequence([f"replacement {i}" for i in range(3)])

{'a': 1,
 'b': 'replacement 0',
 'c': [{'value': 'replacement 1'}, {'value': 'replacement 2'}, {'value': 3}]}

Or provide a mapping based on the selected keypaths:

In [53]:
pz.select(my_nested_object).at_instances_of(jax.Array).set_by_path({
    (jax.tree_util.DictKey(key='b'),): "A",
    (jax.tree_util.DictKey(key='c'), jax.tree_util.SequenceKey(idx=0), jax.tree_util.DictKey(key='value')): "B",
    (jax.tree_util.DictKey(key='c'), jax.tree_util.SequenceKey(idx=1), jax.tree_util.DictKey(key='value')): "C",
})

{'a': 1, 'b': 'A', 'c': [{'value': 'B'}, {'value': 'C'}, {'value': 3}]}

`set_by_path` accepts the same input as is produced by `get_by_path`, which can be convenient for modifying some values and then putting them back, or for taking gradients with respect to only a subset of values:

In [54]:
selection = pz.select(my_nested_object).at_instances_of(jax.Array)
selection.set_by_path(
    {key: f"A JAX array: {value}" for key, value in selection.get_by_path().items()}
)

{'a': 1,
 'b': 'A JAX array: [0 1 2 3 4 5 6 7 8 9]',
 'c': [{'value': 'A JAX array: [ 0  1  2  3  4  5  6  7  8  9 10 11]'},
  {'value': 'A JAX array: [0. 0. 0. 0. 0. 0. 0.]'},
  {'value': 3}]}

In [55]:
def my_loss(obj):
  return obj["a"] + jnp.sum(obj["b"]**2) + jnp.sum(obj["c"][0]["value"])

my_float_object = jax.tree_util.tree_map(lambda leaf: jnp.array(leaf, dtype=jnp.float32), my_nested_object)

# Take gradients w.r.t. non-scalars only, ignoring my_float_object["a"] and my_float_object["c"][2]["value"]
gradient_selection = pz.select(my_float_object).at_instances_of(jax.Array).where(lambda arr: arr.size > 1)

jax.grad(
    # Swap in the version of the values that JAX is taking gradients for:
    lambda vectors_by_path: my_loss(gradient_selection.set_by_path(vectors_by_path))
)(gradient_selection.get_by_path())

OrderedDict([((DictKey(key='b'),),
              <jax.Array([ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16., 18.], dtype=float32)>),
             ((DictKey(key='c'), SequenceKey(idx=0), DictKey(key='value')),
              <jax.Array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)>),
             ((DictKey(key='c'), SequenceKey(idx=1), DictKey(key='value')),
              <jax.Array([0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)])

You can also use `select_and_set_by_path` to infer the selection from the input paths instead:

In [56]:
def my_loss(obj):
  return obj["a"] + jnp.sum(obj["b"]**2) + jnp.sum(obj["c"][0]["value"])

my_float_object = jax.tree_util.tree_map(lambda leaf: jnp.array(leaf, dtype=jnp.float32), my_nested_object)

# Take gradients w.r.t. non-scalars only, ignoring my_float_object["a"] and my_float_object["c"][2]["value"]
# No need to store the gradient selection itself, since it can be inferred from vectors_by_path.
vectors_by_path = pz.select(my_float_object).at_instances_of(jax.Array).where(lambda arr: arr.size > 1).get_by_path()

jax.grad(
    lambda vectors_by_path: my_loss(pz.select(my_float_object).select_and_set_by_path(vectors_by_path))
)(vectors_by_path)

OrderedDict([((DictKey(key='b'),),
              <jax.Array([ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16., 18.], dtype=float32)>),
             ((DictKey(key='c'), SequenceKey(idx=0), DictKey(key='value')),
              <jax.Array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)>),
             ((DictKey(key='c'), SequenceKey(idx=1), DictKey(key='value')),
              <jax.Array([0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)])

### Applying functions to selected values

You can use `.apply` to apply a function to every selected object, similar to `jax.tree_util.tree_map` except that it applies to the selected subtrees rather than to the leaves:

In [57]:
pz.select(my_nested_object).at_instances_of(jax.Array).apply(lambda x: x**2 + 100)

{'a': 1,
 'b': <jax.Array([100, 101, 104, 109, 116, 125, 136, 149, 164, 181], dtype=int32)>,
 'c': [{'value': <jax.Array([100, 101, 104, 109, 116, 125, 136, 149, 164, 181, 200, 221], dtype=int32)>},
  {'value': <jax.Array([100., 100., 100., 100., 100., 100., 100.], dtype=float32)>},
  {'value': 3}]}

You can also do `apply(fn, with_keypath=True)` to get access to the key paths as well:

In [58]:
(
    pz.select(my_nested_object)
    .at_instances_of(jax.Array)
    .apply(lambda key, value: f"key={key}, value={value}", with_keypath=True)
)

{'a': 1,
 'b': "key=(DictKey(key='b'),), value=[0 1 2 3 4 5 6 7 8 9]",
 'c': [{'value': "key=(DictKey(key='c'), SequenceKey(idx=0), DictKey(key='value')), value=[ 0  1  2  3  4  5  6  7  8  9 10 11]"},
  {'value': "key=(DictKey(key='c'), SequenceKey(idx=1), DictKey(key='value')), value=[0. 0. 0. 0. 0. 0. 0.]"},
  {'value': 3}]}

By default, applying a function also removes the selection. If you want to do further processing, you can pass `keep_selected=True` to replace the values but keep them selected:

In [59]:
(
    pz.select(my_nested_object)
    .at_instances_of(jax.Array)
    .apply(lambda key, value: f"key={key}, value={value}", with_keypath=True, keep_selected=True)
)

Selection(
  selected_by_path=OrderedDict({
    (DictKey(key='b'),):
      "key=(DictKey(key='b'),), value=[0 1 2 3 4 5 6 7 8 9]",
    (DictKey(key='c'), SequenceKey(idx=0), DictKey(key='value')):
      "key=(DictKey(key='c'), SequenceKey(idx=0), DictKey(key='value')), value=[ 0  1  2  3  4  5  6  7  8  9 10 11]",
    (DictKey(key='c'), SequenceKey(idx=1), DictKey(key='value')):
      "key=(DictKey(key='c'), SequenceKey(idx=1), DictKey(key='value')), value=[0. 0. 0. 0. 0. 0. 0.]",
  }),
  remainder={'a': 1, 'b': SelectionHole(path=(DictKey(key='b'),)), 'c': [{'value': SelectionHole(path=(DictKey(key='c'), SequenceKey(idx=0), DictKey(key='value')))}, {'value': SelectionHole(path=(DictKey(key='c'), SequenceKey(idx=1), DictKey(key='value')))}, {'value': 3}]},
)

Sometimes, it's also useful to pass the index of the selected node relative to the *selection* (e.g. "this is the third selected node") instead of the absolute keypath. For this, theres `.apply_with_selected_index`:

In [60]:
(
    pz.select(my_nested_object)
    .at_instances_of(jax.Array)
    .apply_with_selected_index(lambda index, value: f"index={index}, value={value}", keep_selected=True)
)

Selection(
  selected_by_path=OrderedDict({
    (DictKey(key='b'),):
      'index=0, value=[0 1 2 3 4 5 6 7 8 9]',
    (DictKey(key='c'), SequenceKey(idx=0), DictKey(key='value')):
      'index=1, value=[ 0  1  2  3  4  5  6  7  8  9 10 11]',
    (DictKey(key='c'), SequenceKey(idx=1), DictKey(key='value')):
      'index=2, value=[0. 0. 0. 0. 0. 0. 0.]',
  }),
  remainder={'a': 1, 'b': SelectionHole(path=(DictKey(key='b'),)), 'c': [{'value': SelectionHole(path=(DictKey(key='c'), SequenceKey(idx=0), DictKey(key='value')))}, {'value': SelectionHole(path=(DictKey(key='c'), SequenceKey(idx=1), DictKey(key='value')))}, {'value': 3}]},
)

### Manipulating selected elements of lists and tuples

When the selected nodes are elements of a list or tuple, there are a few other options for selector manipulation. For instance, you can insert values before or after the selected nodes:

In [61]:
(
    pz.select({"a": list(range(10)), "b": list(range(10, 20))})
      .at_instances_of(int)
      .where(lambda x: x % 4 == 0)
      .insert_before("before a multiple of 4")
)

{'a': ['before a multiple of 4',
  0,
  1,
  2,
  3,
  'before a multiple of 4',
  4,
  5,
  6,
  7,
  'before a multiple of 4',
  8,
  9],
 'b': [10,
  11,
  'before a multiple of 4',
  12,
  13,
  14,
  15,
  'before a multiple of 4',
  16,
  17,
  18,
  19]}

In [62]:
(
    pz.select({"a": list(range(10)), "b": list(range(10, 20))})
      .at_instances_of(int)
      .where(lambda x: x % 4 == 0)
      .insert_after("after a multiple of 4")
)

{'a': [0,
  'after a multiple of 4',
  1,
  2,
  3,
  4,
  'after a multiple of 4',
  5,
  6,
  7,
  8,
  'after a multiple of 4',
  9],
 'b': [10,
  11,
  12,
  'after a multiple of 4',
  13,
  14,
  15,
  16,
  'after a multiple of 4',
  17,
  18,
  19]}

You can also just remove the selected nodes:

In [63]:
(
    pz.select({"a": list(range(10)), "b": list(range(10, 20))})
      .at_instances_of(int)
      .where(lambda x: x % 4 == 0)
      .remove_from_parent()
)

{'a': [1, 2, 3, 5, 6, 7, 9], 'b': [10, 11, 13, 14, 15, 17, 18, 19]}

If you want more control, or if you want to use the selected value to determine what to insert, you can use the method `.apply_and_inline`. This works like `.apply`, except that your function should return a *sequence* of values, and those values will be spliced into the original list or tuple in the same position as the original selected nodes.

In [64]:
(
    pz.select({"a": list(range(10)), "b": list(range(10, 20))})
      .at_instances_of(int)
      .where(lambda x: x % 4 == 0)
      .apply_and_inline(lambda x: ["before", f"the value was {x}", "after"])
)

{'a': ['before',
  'the value was 0',
  'after',
  1,
  2,
  3,
  'before',
  'the value was 4',
  'after',
  5,
  6,
  7,
  'before',
  'the value was 8',
  'after',
  9],
 'b': [10,
  11,
  'before',
  'the value was 12',
  'after',
  13,
  14,
  15,
  'before',
  'the value was 16',
  'after',
  17,
  18,
  19]}

## Taking advantage of key paths

Many of the selector functions allow you to use the PyTree path to each selected in addition to its value:

In [65]:
# Selecting by keypath directly
pz.select(my_nested_object).at_keypaths([
    (jax.tree_util.DictKey(key='c'), jax.tree_util.SequenceKey(idx=2), jax.tree_util.DictKey(key='value')),
    (jax.tree_util.DictKey(key='c'), jax.tree_util.SequenceKey(idx=0)),
])

Selection(
  selected_by_path=OrderedDict({
    (DictKey(key='c'), SequenceKey(idx=0)):
      {
        'value': <jax.Array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int32)>,
      },
    (DictKey(key='c'), SequenceKey(idx=2), DictKey(key='value')):
      3,
  }),
  remainder={
    'a': 1,
    'b': <jax.Array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>,
    'c': [SelectionHole(path=(DictKey(key='c'), SequenceKey(idx=0))), {'value': <jax.Array([0., 0., 0., 0., 0., 0., 0.], dtype=float32)>}, {'value': SelectionHole(path=(DictKey(key='c'), SequenceKey(idx=2), DictKey(key='value')))}],
  },
)

In [66]:
# Selecting by keypath and value
(
    pz.select(my_nested_object)
    .at_subtrees_where(
        lambda path, subtree: len(path) == 3 and isinstance(subtree, jax.Array),
        with_keypath=True)
)

Selection(
  selected_by_path=OrderedDict({
    (DictKey(key='c'), SequenceKey(idx=0), DictKey(key='value')):
      <jax.Array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int32)>,
    (DictKey(key='c'), SequenceKey(idx=1), DictKey(key='value')):
      <jax.Array([0., 0., 0., 0., 0., 0., 0.], dtype=float32)>,
  }),
  remainder={
    'a': 1,
    'b': <jax.Array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>,
    'c': [{'value': SelectionHole(path=(DictKey(key='c'), SequenceKey(idx=0), DictKey(key='value')))}, {'value': SelectionHole(path=(DictKey(key='c'), SequenceKey(idx=1), DictKey(key='value')))}, {'value': 3}],
  },
)

In [67]:
# Setting values by keypath
pz.select(my_nested_object).at_instances_of(jax.Array).set_by_path(lambda path: str(path))

{'a': 1,
 'b': "(DictKey(key='b'),)",
 'c': [{'value': "(DictKey(key='c'), SequenceKey(idx=0), DictKey(key='value'))"},
  {'value': "(DictKey(key='c'), SequenceKey(idx=1), DictKey(key='value'))"},
  {'value': 3}]}

In [68]:
# Setting values by keypath and original value
pz.select(my_nested_object).at_instances_of(jax.Array).apply(
    lambda path, value: str((path, value)), with_keypath=True
)

{'a': 1,
 'b': "((DictKey(key='b'),), Array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32))",
 'c': [{'value': "((DictKey(key='c'), SequenceKey(idx=0), DictKey(key='value')), Array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11], dtype=int32))"},
  {'value': "((DictKey(key='c'), SequenceKey(idx=1), DictKey(key='value')), Array([0., 0., 0., 0., 0., 0., 0.], dtype=float32))"},
  {'value': 3}]}

You can use `jax.tree_util.keystr` or `pz.pretty_keystr` to turn keypaths into readable strings (sometimes useful for referring to parts of a tree by name):

In [69]:
mlp = pz.nn.initialize_parameters(
    simple_mlp.MLP.from_config(feature_sizes=[32, 64, 64, 16]),
    jax.random.PRNGKey(0),
)
mlp

MLP( # Sequential
  sublayers=[
    Affine( # Sequential
      sublayers=[
        Linear(weights=Parameter(value=<NamedArray float32(| features:32, features_out:64) ≈-0.0026 ±0.15 [≥-0.25, ≤0.25] nonzero:2_048 (wrapping jax.Array)>, name='Affine_0.Linear.weights'), in_axis_names=('features',), out_axis_names=('features_out',)),
        RenameAxes(old=('features_out',), new=('features',)),
        AddBias(bias=Parameter(value=<NamedArray float32(| features:64) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:64 (wrapping jax.Array)>, name='Affine_0.AddBias.bias'), new_axis_names=()),
      ],
    ),
    Elementwise(fn=jax.nn.relu),
    Affine( # Sequential
      sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| features:64, features_out:64) ≈-0.0014 ±0.13 [≥-0.22, ≤0.22] nonzero:4_096 (wrapping jax.Array)>, name='Affine_1.Linear.weights'), in_axis_names=('features',), out_axis_names=('features_out',)), RenameAxes(old=('features_out',), new=('features',)), AddBias(bias=Parameter(value=<NamedArray float32(| features:64) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:64 (wrapping jax.Array)>, name='Affine_1.AddBias.bias'), new_axis_names=())],
    ),
    Elementwise(fn=jax.nn.relu),
    Affine( # Sequential
      sublayers=[Linear(weights=Parameter(value=<NamedArray float32(| features:64, features_out:16) ≈-0.004 ±0.16 [≥-0.27, ≤0.27] nonzero:1_024 (wrapping jax.Array)>, name='Affine_2.Linear.weights'), in_axis_names=('features',), out_axis_names=('features_out',)), RenameAxes(old=('features_out',), new=('features',)), AddBias(bias=Parameter(value=<NamedArray float32(| features:16) ≈0.0 ±0.0 [≥0.0, ≤0.0] zero:16 (wrapping jax.Array)>, name='Affine_2.AddBias.bias'), new_axis_names=())],
    ),
  ],
)

In [70]:
[
  key for key in
  pz.select(mlp).at_instances_of(jax.Array).selected_by_path.keys()
]

[(GetAttrKey(name='sublayers'),
  SequenceKey(idx=0),
  GetAttrKey(name='sublayers'),
  SequenceKey(idx=0),
  GetAttrKey(name='weights'),
  GetAttrKey(name='value'),
  GetAttrKey(name='data_array')),
 (GetAttrKey(name='sublayers'),
  SequenceKey(idx=0),
  GetAttrKey(name='sublayers'),
  SequenceKey(idx=2),
  GetAttrKey(name='bias'),
  GetAttrKey(name='value'),
  GetAttrKey(name='data_array')),
 (GetAttrKey(name='sublayers'),
  SequenceKey(idx=2),
  GetAttrKey(name='sublayers'),
  SequenceKey(idx=0),
  GetAttrKey(name='weights'),
  GetAttrKey(name='value'),
  GetAttrKey(name='data_array')),
 (GetAttrKey(name='sublayers'),
  SequenceKey(idx=2),
  GetAttrKey(name='sublayers'),
  SequenceKey(idx=2),
  GetAttrKey(name='bias'),
  GetAttrKey(name='value'),
  GetAttrKey(name='data_array')),
 (GetAttrKey(name='sublayers'),
  SequenceKey(idx=4),
  GetAttrKey(name='sublayers'),
  SequenceKey(idx=0),
  GetAttrKey(name='weights'),
  GetAttrKey(name='value'),
  GetAttrKey(name='data_array')),
 (GetAttrKey(name='sublayers'),
  SequenceKey(idx=4),
  GetAttrKey(name='sublayers'),
  SequenceKey(idx=2),
  GetAttrKey(name='bias'),
  GetAttrKey(name='value'),
  GetAttrKey(name='data_array'))]

In [71]:
[
  jax.tree_util.keystr(key) for key in
  pz.select(mlp).at_instances_of(jax.Array).selected_by_path.keys()
]

['.sublayers[0].sublayers[0].weights.value.data_array',
 '.sublayers[0].sublayers[2].bias.value.data_array',
 '.sublayers[2].sublayers[0].weights.value.data_array',
 '.sublayers[2].sublayers[2].bias.value.data_array',
 '.sublayers[4].sublayers[0].weights.value.data_array',
 '.sublayers[4].sublayers[2].bias.value.data_array']

In [72]:
[
  pz.pretty_keystr(key, mlp) for key in
  pz.select(mlp).at_instances_of(jax.Array).selected_by_path.keys()
]

['MLP.sublayers[0]/Affine.sublayers[0]/Linear.weights/Parameter.value/NamedArray.data_array',
 'MLP.sublayers[0]/Affine.sublayers[2]/AddBias.bias/Parameter.value/NamedArray.data_array',
 'MLP.sublayers[2]/Affine.sublayers[0]/Linear.weights/Parameter.value/NamedArray.data_array',
 'MLP.sublayers[2]/Affine.sublayers[2]/AddBias.bias/Parameter.value/NamedArray.data_array',
 'MLP.sublayers[4]/Affine.sublayers[0]/Linear.weights/Parameter.value/NamedArray.data_array',
 'MLP.sublayers[4]/Affine.sublayers[2]/AddBias.bias/Parameter.value/NamedArray.data_array']